In [18]:
from Extractor_Code.csv_extractor import CSVExtractor
from Extractor_Code.database_connector import DatabaseConnector
from Extractor_Code.api_extractor import APIExtractor
from Extractor_Code.s3_extractor import PublicS3Extractor
# from Extractor_Code.main_extractor import MainExtractor
from Extractor_Code.json_extractor import JSONExtractor


In [19]:
from dotenv import load_dotenv
import yaml
import os

In [ ]:
# adjust this path to where your .env actually lives
load_dotenv(r"C:\Users\ASUS\Desktop\FDE\.env")

config_path = r"C:\Users\ASUS\Desktop\FDE\config.yaml"
with open(config_path) as f:
    raw = f.read()

In [21]:
# expands ${DB_HOST}, ${DB_USER}, etc. using os.environ
expanded = os.path.expandvars(raw)
config = yaml.safe_load(expanded)

# sanity check: should be a dict-of-dicts
print("CONFIG:", config)
print("TYPE of config['database']:", type(config['database']))

CONFIG: {'database': {'host': 'localhost', 'database': 'FDE', 'user': 'postgres', 'password': 'newpassword', 'port': 5432}, 's3': {'bucket_name': 'firstworkshop', 'region': 'us-east-1', 'files': {'JSON/products.json': 'lnd_products_json', 'JSON/sales.json': 'lnd_sales_json', 'CSV/customers.csv': 'lnd_customers_csv', 'CSV/products.csv': 'lnd_products_csv', 'CSV/sales.csv': 'lnd_sales_csv'}}, 'api': {'endpoints': {'https://dummyjson.com/products': 'lnd_products_api', 'https://dummyjson.com/users': 'lnd_users_api'}}}
TYPE of config['database']: <class 'dict'>


In [22]:
# 3) instantiate your connector with that dict
db = DatabaseConnector(config)

In [23]:
# 4) now test psycopg2
print("\n--- psycopg2 test ---")
conn = db.get_connection()
cur  = conn.cursor()

# Test1: Get PostgreSQL version
cur.execute("SELECT version();")
print("Postgres version:", cur.fetchone()[0])


# 2. List all tables in landing schema
cur.execute("""
    SELECT table_name
      FROM information_schema.tables
     WHERE table_schema = 'landing';
""")
print("Tables in public schema:", cur.fetchall())

# 3. show the structure of one of the tables
cur.execute("""
    SELECT column_name, data_type
      FROM information_schema.columns
     WHERE table_schema = 'landing' 
    AND table_name = 'lnd_customers_csv';
""")
print("Table structure:", cur.fetchall())

cur.close()
conn.close()


--- psycopg2 test ---
Postgres version: PostgreSQL 17.5 on x86_64-windows, compiled by msvc-19.44.35209, 64-bit
Tables in public schema: [('lnd_products_api',), ('lnd_users_api',), ('lnd_products_json',), ('lnd_sales_json',), ('lnd_customers_csv',), ('lnd_products_csv',), ('lnd_sales_csv',)]
Table structure: [('id', 'integer'), ('customer_key', 'character varying'), ('gender', 'character varying'), ('name', 'character varying'), ('city', 'character varying'), ('state_code', 'character varying'), ('state', 'character varying'), ('zip_code', 'character varying'), ('country', 'character varying'), ('continent', 'character varying'), ('birthday', 'date'), ('loaded_at', 'timestamp without time zone')]


In [24]:
# 5) and test SQLAlchemy engine
from sqlalchemy import text, MetaData, Table, inspect
print("\n--- SQLAlchemy engine test ---")
engine = db.get_engine()

with engine.connect() as conn2:
    dbname = conn2.execute(text("SELECT current_database();")).scalar()
    print("Connected to database:", dbname)

my_table = Table('lnd_customers_csv', MetaData(), autoload_with=engine,schema='landing')
print(my_table)

# 2. Create an Inspector
inspector = inspect(engine)
# 3. Pull column info for landing.your_table
columns = inspector.get_columns('lnd_customers_csv', schema='landing')
print("Columns in lnd_customers_csv:", columns)


--- SQLAlchemy engine test ---
Connected to database: FDE
landing.lnd_customers_csv
Columns in lnd_customers_csv: [{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': "nextval('landing.lnd_customers_csv_id_seq'::regclass)", 'autoincrement': True, 'comment': None}, {'name': 'customer_key', 'type': VARCHAR(length=50), 'nullable': True, 'default': None, 'autoincrement': False, 'comment': None}, {'name': 'gender', 'type': VARCHAR(length=20), 'nullable': True, 'default': None, 'autoincrement': False, 'comment': None}, {'name': 'name', 'type': VARCHAR(length=255), 'nullable': True, 'default': None, 'autoincrement': False, 'comment': None}, {'name': 'city', 'type': VARCHAR(length=100), 'nullable': True, 'default': None, 'autoincrement': False, 'comment': None}, {'name': 'state_code', 'type': VARCHAR(length=50), 'nullable': True, 'default': None, 'autoincrement': False, 'comment': None}, {'name': 'state', 'type': VARCHAR(length=100), 'nullable': True, 'default': None, 'autoincreme